In [ ]:
https://github.com/DirkEngfer/PandasTests
Air pollution in the EU, programmer: Dirk Engfer, Germany
Provided Input data file: airpollution2012.csv

------------------------------
Original source of Input Data:
------------------------------
Creator: EEA - European Environment Agency, Copenhagen
Title: National emissions reported to the Convention on Long-range Transboundary Air Pollution (LRTAP Convention)
Copyright: European Environment Agency (EEA) is the owner of copyrights and database rights
Copyright notice: Information, documents and material available on this website and for which the EEA holds the rights of use are public and may be re-used without prior permission, free of charge, for commercial or non-commercial purposes, provided that the EEA is always acknowledged as the original source of the material and that the original meaning or message of the content is not distorted. Such acknowledgment must be included in each copy of the material. The re-use of the content on the EEA website covers the reproduction, adaptation and/or distribution, irrespective of the means and/or the format used. The re-use of certain data may be subject to different conditions, and if so the item concerned is accompanied by a copyright mark or other mention of the specific conditions relating to it. The above mentioned permissions do not apply to content supplied by third parties. Therefore, for documents where the copyright lies with a third party, permission for reproduction must be obtained from the copyright holder.
Copyright source: https://www.eea.europa.eu/legal/copyright
Download date: 14-May-2020
License notice: https://creativecommons.org/licenses/by/2.5/dk/deed.en_GB 
Data download source: https://www.eea.europa.eu/data-and-maps/data/national-emissions-reported-to-the-convention-on-long-range-transboundary-air-pollution-lrtap-convention-13
Disclaimer notice: No warranties are given. The license may not give you all of the permissions necessary for your intended use. For example, other rights such as publicity, privacy, or moral rights may limit how you use the material.
(Disclaimer source: https://creativecommons.org/licenses/by/2.5/dk/deed.en_GB)


In [1]:
import os, numpy as np
import pandas as pd
homedir = os.getenv('HOME')

datapath = os.path.join(homedir, 'Dokumente','python-apps','tensorflow', 'eu_air_pollution_data')
datafile = 'airpollution2012.csv'

indatapath = os.path.join(datapath,datafile)

In [ ]:
In the following, we do a test how to translate SAS code "if first.statistics_year then count_by_year = 1; else
count_by_year + 1;" In Pandas we can employ method cumcount() to accomplish this. As cumcount() starts with 0 we
additionally put method add(1) in place to count up by 1:

In [175]:
c = lambda a : (  a.replace(',', '.')  )

df = pd.read_csv(indatapath, skiprows=10, header=0, sep=',',usecols=[0,1,2,3,5,7,9,13,14], converters={'statistic_value':c})
df = df.loc[(df.city_name.isin(['Kiel', 'Lübeck']))]
df['statistic_value'] = df['statistic_value'].astype(np.float)
df.sort_values(by=['city_name', 'statistics_year'], inplace=True)
df.drop(['type_of_station', 'station_type_of_area', 'component_caption', 'measurement_unit', 'country_name', 'station_european_code'], axis=1, inplace=True)
#df.reset_index(drop=True, inplace=True)
#df.index = df.statistics_year
#df.drop(columns='statistics_year', inplace=True)

df1 = df.loc[df['city_name'] == 'Kiel', :].copy()
df1.drop(columns='city_name', axis=1, inplace=True)
df1.rename(columns={"statistic_value": "Kiel"}, inplace=True)
df2 = df.loc[df['city_name'] == 'Lübeck', :].copy()
df2.drop(columns='city_name', axis=1, inplace=True)
df2.rename(columns={"statistic_value": "Lübeck"}, inplace=True)

df3 = df1.loc[:, ['Kiel', 'statistics_year']]
df3['counter'] = range(0, len(df3))
kiel_gr = df3.groupby(df3['statistics_year'], as_index=False)
first = kiel_gr.first()
first['isfirst'] = 1
first.drop(columns=['statistics_year', 'Kiel'], axis=1, inplace=True)
df3 = df3.merge(first, how='left', on='counter')
df3['count_by_year'] = df3.groupby(df3['statistics_year']).cumcount().add(1)
df3


,Kiel,statistics_year,counter,isfirst,count_by_year
0,36.729,2001,0,1.0,1
1,38.750,2002,1,1.0,1
2,47.000,2003,2,1.0,1
3,43.875,2003,3,NaN,2
4,30.304,2004,4,1.0,1
5,31.421,2005,5,1.0,1
6,32.583,2006,6,1.0,1
7,41.000,2007,7,1.0,1
8,34.250,2007,8,NaN,2
9,35.670,2008,9,1.0,1


In [ ]:
"""   """